### Welcome to MMTracking!
In this tutorial, you will learn:
+ to perform inference with MMTracking.
+ to train a new dataset and visiualize the results

### Install MMTracking

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

In [ ]:
# install PyTorch
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html

# install MMDetection
!pip install mmdet

# clone the MMTracking repository
!git clone https://github.com/open-mmlab/mmtracking.git
%cd mmtracking

!pip install -r requirements/build.txt
!pip install -e .

In [ ]:
from mmcv import collect_env
collect_env()

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check MMTracking installation
import mmtrack
print(mmtrack.__version__)

In [ ]:
# Run some inference demos
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmtracking/vid/selsa/selsa_faster_rcnn_r50_dc5_1x_imagenetvid/selsa_faster_rcnn_r50_dc5_1x_imagenetvid_20201227_204835-2f5a4952.pth -P checkpints
!wget -c https://download.openmmlab.com/mmtracking/sot/siamese_rpn/siamese_rpn_r50_1x_lasot/siamese_rpn_r50_1x_lasot_20211203_151612-da4b3c66.pth -P checkpints
!wget -c https://download.openmmlab.com/mmtracking/vis/masktrack_rcnn/masktrack_rcnn_r50_fpn_12e_youtubevis2019/masktrack_rcnn_r50_fpn_12e_youtubevis2019_20211022_194830-6ca6b91e.pth -P checkpoints

In [ ]:
import mmcv
import tempfile
from mmtrack.apis import inference_mot, init_model
config_file = '../configs/mot/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py'
# build the model from a config file and a checkpoint file
model = init_model(config, device='cuda:0')
input_video = '../demo/demo.mp4'
output_video = './demo/mot.mp4'
imgs = mmcv.VideoReader(video)
prog_bar = mmcv.ProgressBar(len(imgs))
# test and show/save the images
for i, img in enumerate(imgs):
    result = inference_mot(model, img, frame_id=i)
    model.show_result(
        img,
        result,
        score_thr=0,
        show=True,
        wait_time=int(1000. / imgs.fps),
        out_file=None,
        backend='plt')
    prog_bar.update()